Import packages:

In [1]:
import numpy as np
import pandas as pd

Check pandas version (to doublecheck if loaded):

In [2]:
pd.__version__

'2.3.3'

### Importing and inspecting data

Import data from CSV: `inventory.csv`

In [4]:
df = pd.read_csv("inventory.csv") # df is the typical abbreviation for dataframe! Careful with the working directory!
df

,Substance,bottle_volume,remaining_amount,price
0,Acetone,500,120.0,8.5
1,Acetone,1000,840.0,14.0
2,Ethanol,1000,600.0,12.0
3,Ethanol,500,120.0,6.8
4,Ethanol,250,NaN,3.5
5,Sodium chloride,250,200.0,1.2
6,Hydrochloric acid,500,320.0,9.9
7,Acetic acid,100,NaN,4.6
8,Methanol,1000,910.0,11.5
9,Sodium hydroxide,500,240.0,7.2


Get basic information about dataset (info, shape, size):

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Substance         11 non-null     object 
 1   bottle_volume     11 non-null     int64  
 2   remaining_amount  9 non-null      float64
 3   price             11 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 484.0+ bytes


Summary statistics: describe!

In [6]:
df.describe()

,bottle_volume,remaining_amount,price
count,11.000000,9.000000,11.000000
mean,554.545455,398.888889,7.854545
std,316.586912,305.795865,3.855480
min,100.000000,120.000000,1.200000
25%,375.000000,200.000000,5.700000
50%,500.000000,240.000000,7.200000
75%,750.000000,600.000000,10.700000
max,1000.000000,910.000000,14.000000


### Indexing & selection
Columns can be called by simply selecting the column name from the dataframe, e.g. `df["name"]`. Multiple columns can be listed! This can also be used to create new columns.



In [7]:
df["price"]

0      8.5
1     14.0
2     12.0
3      6.8
4      3.5
5      1.2
6      9.9
7      4.6
8     11.5
9      7.2
10     7.2
Name: price, dtype: float64

Entries can be localized by their coordinates using `loc` (explicit indices) and `iloc` (implicit indices). Can be used to change single values.

In [8]:
df.iloc[0,0]

'Acetone'

This can also be used for slicing! e.g. `df.iloc[startrow:endrow:interval, startcolumn:endcolumn:interval]` (endpoints not included! if one of the numbers not given: default = from start to end with interval 1, i.e. every entry).

In [9]:
df_small = df.iloc[0:3:,::]
df_small

,Substance,bottle_volume,remaining_amount,price
0,Acetone,500,120.0,8.5
1,Acetone,1000,840.0,14.0
2,Ethanol,1000,600.0,12.0


Columns are accessed using the `columns` attribute, rows using the `index` attribute (can be used for renaming the whole axis).

In [10]:
df_small.columns = ["A", "B", "C", "D"]
df_small

,A,B,C,D
0,Acetone,500,120.0,8.5
1,Acetone,1000,840.0,14.0
2,Ethanol,1000,600.0,12.0


Creating new columns, e.g. price per volume:

In [ ]:
df["price_per_vol"] = df["price"]/df["bottle_volume"] # equivalent to:
# new_df = df.assign(price_per_vol = df["price"]/df["bottle_volume"]) 
df

Selecting with certain condition: Boolean masks for selecting entries:

In [ ]:
df.loc[df["Substance"] == "Ethanol"]

### Data cleaning & processing

To delete rows and columns, `drop` can be used (specifying the axis!):

In [ ]:
# Drop the last column on a copy of teh original df
df_reduced = df.copy().drop(columns=["price_per_vol"], axis=1)
df_reduced

Most important use cases: handle missing values and duplicates.

Detect missing values: `isna()` - best used in combination with `sum()` or `any()`. Can be used on df or columns.

In [ ]:
df.isna().any()

If desired, rows with missing values can be dropped with `dropna()` (e.g. if crucial value). Good practice: Documentation! And keep original df as copy!

In [ ]:
df2 = df.copy().dropna()
df2

Duplicates can be detected by `duplicated()`. Duplicates can be dropped using `drop_duplicates()`. Good practice: Documentation and working on copies!

In [ ]:
df.duplicated().sum()

Both cases, duplicates and missing values, can be treated with other options (see documentation)! Or completely different approaches may be valid, such as filling or flagging.

Types (info!) can be redefined using `astype`:

In [ ]:
print(df.info())
df["bottle_volume"] = df["bottle_volume"].astype(float)
df.info()

### Grouping & aggregation

`groupby` is an important example of a split-apply-combine approach: first, entries in a specified column will be grouped together, then a function is applied (e.g. an aggregate such as mean, std.dev., sum) to all numerical values in each group, then the groups are recombined to give another dataframe.

Single columns can be selected from the groups to which the aggregates will be applied. Use `agg()` to mix different aggregates for different columns (refer to documentation).

In [ ]:
df.groupby("Substance").mean()

### Merging and joining datasets

`pd.merge` and `pd.concat` are very important functions to combine different Dataframes (or Series).

First load as `df2` the CSV file `chemicals.csv`:

In [ ]:
df2 = pd.read_csv("chemicals.csv")
df2

`pd.concat` mainly appends the dataframes, per default as new rows (can be specified with axis!). `join=` defines if everything is kept ("outer") or only the entries that are existing in both ("inner"). `ignore_index=True` will replace the original indices.

In [ ]:
df_appended = pd.concat((df, df2), join="outer", ignore_index=True)
df_appended

`pd.merge` fuses two dataframes together (specified with `left=` and `right=`) `on` a specified common key, `how` sets the merge type (similar as to the join type with `concat`, but with more options). Merges can be also done on different keys (`left_on=`, `right_on=`). `suffixes` can fix clashes when more columns bear the same name in the two dataframes (see documentaiton).

In [ ]:
df_merged = pd.merge(left=df, right=df2, on="Substance", how="outer")
df_merged

### Reshaping data

`melt` transforms a "wide table" into a long one ("Take many columns and stack them into rows."). Often this is done for visualisation. The basic syntax is: `pd.melt(df, id_vars=..., value_vars=..., var_name=..., value_name=...)`, where...

...`id_vars`: columns to keep as identifiers (stay the same)
...`value_vars`: columns that will be unpivoted (melted into rows)
...`var_name`: name of the new column that will hold the former column names
...`value_name`: name of the new column that will hold the values

`pivot` is the reverse operation, `pivot_table` works in a similar way - refer to documentation.

In [ ]:
# not the most meaningful example... (better one see dht_data)
print(df)
df_molten = df.melt(
    id_vars="Substance", 
    value_vars=["bottle_volume", "remaining_amount", "price", "price_per_vol"],
    var_name="container_property",
    value_name="value"
    )
df_molten